In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
SOS_token = 0
EOS_token = 1
UNKNOWN_token = 2


# 읽어들인 소스와 타깃 문서의 전체 단어를 관리하기 위한 클래스 (※1)
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2:"UNKNOWN"}
        self.n_words = 3  # Count SOS and EOS and UNKNOWN

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

    def getWordIndex(self, word):
        if word in self.word2index:
            return self.word2index[word]
        else:
            return 2 # UNKNOWN


In [ ]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


# 불필요한 문자를 삭제한다 (※2)
def normalizeString(s):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣 ^☆; ^a-zA-Z.!?]+')
    s = hangul.sub('', s)
    return s

In [ ]:
# 소스 파일과 타깃 파일을 읽는다 (※3)
def readLangs():
    print("Reading lines...")

    # 소스 파일과 타깃 파일을 줄단위로 읽어들인다 (※4)
    inputs = open('/content/drive/MyDrive/university/nlp_cv/laundry_input.txt', encoding='utf-8').read().strip().split('\n')
    outputs = open('/content/drive/MyDrive/university/nlp_cv/laundry_output.txt', encoding='utf-8').read().strip().split('\n')

    inputs = [normalizeString(s) for s in inputs]
    outputs = [normalizeString(s) for s in outputs]

    input_lang = Lang('input')
    output_lang = Lang('output')

    # 소스와 타깃 문장을 짝을지어 pairs 변수에 추가한다 (※5)
    pairs = []
    for i in range(len(inputs)):
        pairs.append([inputs[i], outputs[i]])

    return input_lang, output_lang, pairs

In [ ]:
# 최대 문장의 텍스트 길이 (※6)
MAX_LENGTH = 100

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
# 데이터 준비  클래스 (※7)
def prepareData():
    input_lang, output_lang, pairs = readLangs()
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


# 데이터 준비 (※8)
input_lang, output_lang, pairs = prepareData()
print(random.choice(pairs))

Reading lines...
Read 134 sentence pairs
Counting words...
Counted words:
input 190
output 161
['모피 세작가능', '가능합니다. 가죽 모피류는 소재에 따라 검품 후 수작업 특수세탁으로 처리합니다.']


In [ ]:
# 데이터 인코딩 (※9)
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
# 데이터 디코딩 (※10)
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
# 어텐션 클래스 (※11)
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.getWordIndex(word) for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
teacher_forcing_ratio = 0.5


# 학습 수행 (※12)
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))


In [ ]:
# 전체 학습 단위수행 (※13)
def trainIters(encoder, decoder, n_iters, print_every=2000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
# 평가하는 함수 (※14)
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]


In [ ]:
# 임의 데이터 테스트 (※15)
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 2000, print_every=200)

############################################################



0m 11s (- 1m 41s) (200 10%) 3.0298
0m 18s (- 1m 13s) (400 20%) 2.6294
0m 23s (- 0m 55s) (600 30%) 2.1828
0m 29s (- 0m 44s) (800 40%) 1.7418
0m 35s (- 0m 35s) (1000 50%) 1.3226
0m 41s (- 0m 27s) (1200 60%) 1.0072
0m 48s (- 0m 20s) (1400 70%) 0.4614
0m 54s (- 0m 13s) (1600 80%) 0.3716
1m 1s (- 0m 6s) (1800 90%) 0.1100
1m 7s (- 0m 0s) (2000 100%) 0.0886


In [ ]:
evaluateRandomly(encoder1, attn_decoder1)
output_words, attentions = evaluate(
    encoder1, attn_decoder1, "바지 수선하고 싶어요.")
plt.matshow(attentions.numpy())


> 가죽 세탁가능
= 가능합니다. 가죽 모피류는 소재에 따라 검품 후 수작업 특수세탁으로 처리합니다.
< 가능합니다. 가죽 모피류는 소재에 따라 검품 후 수작업 특수세탁으로 처리합니다. <EOS>

> 세탁 오래걸려?
= 영업일 기준 박 일 소요됩니다. 세탁물이 많이 몰리는 경우 조금 더 소요될 수 있습니다. 빠른 세탁서비스가 필요할 경우 당일세탁 서비스 이용을 요청하시길 바랍니다.
< 영업일 기준 박 일 소요됩니다. 세탁물이 많이 몰리는 경우 조금 더 소요될 수 있습니다. 빠른 세탁서비스가 필요할 경우 당일세탁 서비스 이용을 요청하시길 바랍니다. <EOS>

> 세탁 기간은
= 영업일 기준 박 일 소요됩니다. 세탁물이 많이 몰리는 경우 조금 더 소요될 수 있습니다. 빠른 세탁서비스가 필요할 경우 당일세탁 서비스 이용을 요청하시길 바랍니다.
< 영업일 기준 박 일 소요됩니다. 세탁물이 많이 몰리는 경우 조금 더 소요될 수 있습니다. 빠른 세탁서비스가 필요할 경우 당일세탁 서비스 이용을 요청하시길 바랍니다. <EOS>

> 몇시에 여노?
= 주중에는 오전 시부터 오후 시까지 영업합니다.
< 주중에는 오전 시부터 오후 시까지 영업합니다. <EOS>

> 세탁 얼마나 걸려
= 영업일 기준 박 일 소요됩니다. 세탁물이 많이 몰리는 경우 조금 더 소요될 수 있습니다. 빠른 세탁서비스가 필요할 경우 당일세탁 서비스 이용을 요청하시길 바랍니다.
< 영업일 기준 박 일 소요됩니다. 세탁물이 많이 몰리는 경우 조금 더 소요될 수 있습니다. 빠른 세탁서비스가 필요할 경우 당일세탁 서비스 이용을 요청하시길 바랍니다. <EOS>

> 무인 빨래방 영업시간은
=  무인으로 운영되는 코인빨래방이 같은 층 호에 있으니 세탁물 접수 수거시 활용하시길 바랍니다.
<  무인으로 운영되는 코인빨래방이 같은 층 호에 있으니 세탁물 접수 수거시 활용하시길 바랍니다. <EOS>

> 얼마야
= 와이셔츠는 천원 원피스코트는 만원 니트류는 만천원 모피가죽제품은 만원입니다.
< 와이셔츠는 천원 원피스코트

In [ ]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)



# 원하는 문장 예측 (※16)
evaluateAndShowAttention("문 여는 시간 언제야?")
evaluateAndShowAttention("가격을 알려줄래?")
evaluateAndShowAttention("운동화도 세탁이 되나?")

input = 문 여는 시간 언제야?
output = 주중에는 오전 시부터 오후 시까지 영업합니다. <EOS>
input = 가격을 알려줄래?
output = 와이셔츠는 천원 원피스코트는 만원 니트류는 만천원 모피가죽제품은 만원입니다. <EOS>
input = 운동화도 세탁이 되나?
output = 수거 배달 서비스는 하고 있지 않습니다. <EOS>


<ipython-input-22-3996918382>:9: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels([''] + input_sentence.split(' ') +
<ipython-input-22-3996918382>:11: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels([''] + output_words)
<ipython-input-22-3996918382>:9: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels([''] + input_sentence.split(' ') +
<ipython-input-22-3996918382>:11: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels([''] + output_words)
<ipython-input-22-3996918382>:9: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocato

In [ ]:
# 원하는 문장 예측 (※16)
evaluateAndShowAttention("불좀 켜라")
evaluateAndShowAttention("거실 에어컨 켜라")
evaluateAndShowAttention("창문 열어줘")

input = 불좀 켜라
output = 와이셔츠는 천원 원피스코트는 만원 니트류는 만천원 모피가죽제품은 만원입니다. <EOS>
input = 거실 에어컨 켜라
output = 와이셔츠는 천원 원피스코트는 만원 니트류는 만천원 모피가죽제품은 만원입니다. <EOS>
input = 창문 열어줘
output = 와이셔츠는 천원 원피스코트는 만원 니트류는 만천원 모피가죽제품은 만원입니다. <EOS>


<ipython-input-20-3996918382>:9: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels([''] + input_sentence.split(' ') +
<ipython-input-20-3996918382>:11: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels([''] + output_words)
<ipython-input-20-3996918382>:9: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels([''] + input_sentence.split(' ') +
<ipython-input-20-3996918382>:11: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels([''] + output_words)
<ipython-input-20-3996918382>:9: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocato